In [1]:
import pandas as pd
import numpy as np
import names, json
import random

In [11]:
df = pd.read_csv('data/jcpenney/jcpenney.csv')

In [12]:
df.drop(['sku', 'sale_price', 'category', 'product_url', 'total_number_reviews'], inplace=True, axis=1)
df.rename(columns={'uniq_id': 'id', 'name_title': 'name', 'list_price': 'price', 'average_product_rating': 'rating', 'product_image_urls': 'image', 'Reviews': 'reviews', 'category_tree': 'category'}, inplace=True)

In [13]:
# DELETE DUPLICATED ROWS
to_mantain_index = []
to_mantain_name = []
def deleteRows(row):
    if row['name'] not in to_mantain_name:
        to_mantain_index.append(row.name)
        to_mantain_name.append(row['name'])

df.apply(deleteRows, axis=1)
df = df.loc[to_mantain_index]

In [14]:
# RATING
df['rating'] = df['rating'].str.replace(' out of 5','')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [15]:
df['category'] = df['category'].str.split('|')
df['stock'] = np.random.randint(0, 300, df.shape[0])
df['store'] = 'jcpenney'
df['price'] = [round(random.uniform(0.01, 300), ndigits=2) for i in range(df.shape[0])]

In [16]:
df

,id,name,description,price,category,rating,image,brand,reviews,stock,store
0,b6c0b6bea69c722939585baeac73c13d,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,67.23,"[jcpenney, women, alfred dunner]",4.7,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,"{""review""=>[{""review_1""=>""You never have to wo...",110,jcpenney
6,8ffd0ef4fcaf1a82fb514aba5d20e05b,Alfred Dunner® Feels Like Spring 3/4 Sleeve Le...,"For easygoing style you'll love, wear our stre...",17.98,"[jcpenney, women, view all]",5.0,http://s7d9.scene7.com/is/image/JCPenney/DP022...,Alfred Dunner,"{""review""=>[{""review_1""=>""I loved the quality ...",11,jcpenney
7,4d9337e3c8f974d3c420cdc5c58b3fc3,Alfred Dunner® Feels Like Spring 3/4-Sleeve Le...,Spring is in the air with our 3/4-sleeve leaf ...,54.01,"[jcpenney, women, skechers]",3.0,http://s7d9.scene7.com/is/image/JCPenney/DP033...,Alfred Dunner,"{""review""=>{""review_1""=>""The colors and qualit...",17,jcpenney
8,44f8f8f108c6856acf9630dd1d78516d,Alfred Dunner® Feels Like Spring 3/4-Sleeve Wa...,NaN,265.13,"[jcpenney, , search-results]",5.0,http://s7d9.scene7.com/is/image/JCPenney/DP033...,Alfred Dunner,"{""review""=>[{""review_1""=>""I love the way these...",103,jcpenney
11,5abf9d28e9e0404369ece10807d99d0e,Alfred Dunner® Feels Like Spring Capris,Pull on a pair of our casual capris and be foo...,14.79,"[jcpenney, women, capris & crops]",4.6,http://s7d9.scene7.com/is/image/JCPenney/DP022...,Alfred Dunner,"{""review""=>[{""review_1""=>""These capris are ver...",238,jcpenney
...,...,...,...,...,...,...,...,...,...,...,...
19733,0101ffcd60d3796f8c26a7ab8ab7f3c6,Hoover® WindTunnel® MAX™ Pet Plus Bagless Upri...,The WindTunnel MAX Pet Plus bagless vacuum is ...,199.69,"[jcpenney, gift-registry, vacuums & floorcare]",4.1,http://s7d9.scene7.com/is/image/JCPenney/DP020...,Hoover,"{""review""=>[{""review_1""=>""I had a lighter weig...",232,jcpenney
19735,21cd4107d3fa731e88b0a46dddf38bbb,Hoover® WindTunnel® Pet Bagged Upright Vacuum ...,The Hoover Windtunnel Pet Bagged Upright Vacuu...,3.81,"[jcpenney, for-the-home, vacuums & floorcare]",NaN,http://s7d9.scene7.com/is/image/JCPenney/09006...,Hoover,NaN,252,jcpenney
19738,c3324af6ba51a29b5e60d00b399a6647,Hoover® WindTunnel® Pet Rewind Upright Vacuum ...,This high-performing Hoover WindTunnel upright...,217.63,"[jcpenney, for-the-home, hoover]",NaN,http://s7d9.scene7.com/is/image/JCPenney/09006...,Hoover,NaN,277,jcpenney
19744,ab715e3e199dc0cfcb670f2bed919cd6,Hoover® WindTunnel® Self-Propelled Upright Vac...,The Hoover WindTunnel Self-Propelled Bagged Up...,21.31,"[jcpenney, for-the-home, vacuums & floorcare]",NaN,http://s7d9.scene7.com/is/image/JCPenney/09006...,Hoover,NaN,188,jcpenney


In [ ]:
df.dropna(subset=['name'], inplace=True)

In [7]:
def parseReviews(row):
    try:
        st = row.reviews[13:-3]
        st = st.split('}, {')

        reviews = []
        i = 1
        for s in st:
            string = '"review_' + str(i) + '"=>'
            review = s.replace(string, '')
            review = review[1:-1]
            review = " ".join(review.split())
            i += 1
            aux = {'author': names.get_full_name(), 'rating': np.round(np.random.uniform(0.1, 5.0), 1), 'date': '2018-2-5', 'title': "", 'body': review}
            reviews.append(aux)
        return reviews
    except:
        return []

df['reviews'] = df.apply(parseReviews, axis=1)
df['Id'] = ["products/jcpenney/" + str(x) for x in range(1, len(df) + 1)]
df.drop(columns='id', inplace=True)

In [8]:
df.to_csv('data/jcpenney/products.csv', encoding='utf-8', index=False)

In [9]:
parsed = json.loads(df.to_json(orient="records", default_handler=str))

json_string = json.dumps(parsed, indent=4)
with open('data/output/jcpenney.json', 'w') as outfile:
    outfile.write(json_string)